In [5]:
__author__ = "Sagar Sinha"
__task__ = "Text preprocessing and analysis pipeline" 

In [2]:
from ..\src\features\pipeline import pipeline

SyntaxError: unexpected character after line continuation character (1762639170.py, line 1)

In [1]:
# Check for version consistency of Corpus 
try:
    from collections import Counter, defaultdict 
    import re
    import os
    import json
    import string 
   
    # Import third-party libraries
    import pandas as pd
    
    import warnings 
    warnings.filterwarnings('ignore')
    
    import numpy as np
    from collections import defaultdict
    from tqdm import tqdm 
    from convokit import Corpus, Speaker, Utterance, download
    import convokit
    from convokit.text_processing import TextProcessor, TextCleaner, TextParser
    from convokit.convokitPipeline import ConvokitPipeline
    from convokit import PolitenessStrategies
    from textblob import TextBlob as tb
    import nltk
    from nltk.stem import WordNetLemmatizer 
    import spacy 
    import emot
    import contractions as cm 
    import vaderSentiment
    from googleapiclient import discovery
    from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
   
    
    ### Import langdetect and set seed = 0 to enforce consistent results 
    from langdetect import DetectorFactory, detect
    DetectorFactory.seed = 0 
    
    # The below modules can be implemented after the utterances have been loaded as Pandas Dataframe or Numpy arrays 
    # import sklearn                    
    # from sklearn.preprocessing import FunctionTransformer
    # from sklearn.base import BaseEstimator, TransformerMixin
    # from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion, make_union
    # from sklearn.compose import ColumnTransformer, make_column_transformer

except ImportError as e:
  print(e)

In [2]:
# NLTK libraries and dependencies for preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [4]:
# Convokit requires a SpaCy model to be installed. Run `python -m spacy download MODEL_NAME`
! python -m spacy download en_core_web_md

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [3]:
# Load the spacy language model 
# A custom language model can be loaded. Check 
spacy.load("en_core_web_md")

In [4]:
lemmatizer = WordNetLemmatizer() # Installing WordNet Lemmatizer 
sentiment =  SentimentIntensityAnalyzer()  # Intialize Vader Sentiment Analyzer 
emotion = emot.core.emot()  # Initialize Emoji Object
politeness = PolitenessStrategies() # Politeness Indicators
spacy_nlp = spacy.load('en_core_web_md', disable=['ner']) # Spacy Language Model 

In [6]:
!dir

 Volume in drive C is OS
 Volume Serial Number is 6809-FEE9

 Directory of C:\Sagar Study\ML and Learning\CP Sem-8\Data\Reddit\saved-corpora

19-11-2022  08:53    <DIR>          .
19-11-2022  08:53    <DIR>          ..
21-10-2022  22:49    <DIR>          .ipynb_checkpoints
19-11-2022  08:52            36,489 1.0_conversation_metadata .ipynb
17-08-2022  18:39            37,440 1_0_Adding_Metadata.ipynb
19-11-2022  08:53            50,888 2_0_function_migration_pipeline.ipynb
13-08-2022  10:25           269,413 bot-removal.ipynb
19-11-2022  08:49    <DIR>          fc(12-23)_27p_1
26-08-2022  07:39    <DIR>          lexicons
05-08-2022  13:28    <DIR>          mens-rights-2018
06-08-2022  14:56    <DIR>          mens-rights-2018-transformed
06-08-2022  18:16    <DIR>          mens-rights-2018-transformed-2
09-08-2022  20:20    <DIR>          mens-rights-2018-transformed-3
09-08-2022  21:02    <DIR>          mens-rights-2018-transformed-final
09-08-2022  20:13    <DIR>          transformed

In [13]:
# %history

In [21]:
    subjectivity_clues = []

    with open("lexicons/subjectivity_clues.txt", "r+") as file: 
        for line in file.readlines():
            values = line.split(" ")[2]
            subjectivity_clue = values.split("=")[1]
            subjectivity_clues.append(subjectivity_clue)

In [15]:
# Load the Reddit Corpus 
try:
    CORPUS_PATH = "sanskar_transformed_corpora/tr_fc(12-23)_27p_1"   # Provide the path 
    reddit_corpus = Corpus(CORPUS_PATH)
except Exception as e:
    print(e)

In [17]:
def def_value():
    return 0

In [22]:
# Below-mentioned functions would be chained into a universal pipeline for a combined analysis 
# Only input utterance level tests. Speaker-level and conversation-level tests will be added later as we load the dataframe 
# First load the raw corpus into Communalytic. Get scores from that platform 

def extractHashtags(utterance: str) -> list:
    """ Returns all Twitter hashtags from a tweet

    Store all hashtags from a tweet and store them in a separate column
    """
    hashtags_ls = re.findall("#\w+", utterance)
    return hashtags_ls
 
hashtags = TextProcessor(proc_fn=extractHashtags, output_field="hashtags")  # We pass the required function as a parameter to TextProcessor class. The output would be stored in metadata as "hashtags"

def extractUsernameTags(utterance: str) -> list:
    """ Returns all Username tags from a tweet
    Store all tags from a tweet and store them in a separate column
    """

    username_tags = re.findall("@\w+", utterance)
    return username_tags

reply_to_usernames = TextProcessor(proc_fn=extractUsernameTags, output_field="username_tags") # Output is tsored in metadata as "username_tags"


def cleanReddit(
        text_reddit, newline=True, quote=True,
        bullet_point=True, dates=True, link=True,
        strikethrough=True, spoiler=True, heading=True,
        emoj=True, emoticon=True, condensed=True):
    # Newlines we don't need - only
    """ Cleans reddit utterances"""
    
    if newline:
        text_reddit = re.sub(r'\n+', ' ', text_reddit)
        # Remove the many " " that we replaced in the last step
        text_reddit = text_reddit.strip()
        text_reddit = re.sub(r'\s\s+', ' ', text_reddit)

    # > are for the quoted texts from the main comment or the reply
    if quote:
        text_reddit = re.sub(r'>', '', text_reddit)

    # Bullet points/asterisk are used for markdown like - bold/italic - Could create trouble in parsing? idk
    if bullet_point:
        text_reddit = re.sub(r'\*', '', text_reddit)
        text_reddit = re.sub('&amp;#x200B;', '', text_reddit)

    # []() Link format then we remove both the tag/placeholder and the link
    if link:
        text_reddit = re.sub(r"http\S+", '', text_reddit)
        text_reddit = re.sub(r'\[.*?\]\(.*?\)', '', text_reddit)

    # Strikethrough
    if strikethrough:
        text_reddit = re.sub('~', '', text_reddit)

    # Spoiler, which is used with < less-than (Preserves the text)
    if spoiler:
        text_reddit = re.sub('&lt;', '', text_reddit)
        text_reddit = re.sub(r'!(.*?)!', r'\1', text_reddit)

    # Heading to be removed as there are these markdown style features in reddit too
    if heading:
        text_reddit = re.sub('#', '', text_reddit)

    if emoj:
        # Implement the emoji scheme here
        # Implementing a Naive Emoji Scheme
        # Some associated libraries are EMOT and DEMOJI
        # text_reddit = emoji.demojize(text_reddit).replace(":", "").replace("_", "")
        # Makes more sense for the node feature but might as well import that function here if ready
        pass

    if dates:
        text_reddit = re.sub(r'(\d+/\d+/\d+)', '', text_reddit)

    if emoticon:
        # Implement the emoticon scheme here.
        # Makes more sense for the node feature but might as well import that function here if ready
        pass

    # Needs to be the last step in the process
    # if contractions:
    # text = contractions.fix(text)
    # print("Running")
    return text_reddit
     
clean_text_reddit = TextCleaner(text_cleaner = cleanReddit, replace_text = False, save_original = True)
    
def cleanTwitter(
        text_twitter, urls=True, tags=True,
        newLine=True, ellipsis=True, ampersand=True,
        tilde=True, special_chars=True, dollar=True,
        commas_semicols=True, bracketed_phrases=True, contractions=True,
        quotation_marks=True, greater_than_less_than=True, question_mark_exclaim=True,
        character_encodings=True, trademark=True, condensed=True) -> str:
    """ Clean tweets after extracting all hashtags and username tags
    Not comprehensive enough to capture all idiosyncrasies, but works for most of the time
    """
    if urls:
        url_pattern = "https?:\/\/(www\.)?(\w+)(\.\w+)\/\w*"
        text_twitter = re.sub(url_pattern, "", text_twitter)

    if tags:
        text_twitter = re.sub("@\w+", "", text_twitter)

    # Remove "\n". One or more occurrences
    if newLine:
        # Replacing single occurrences of '\n' with ''
        # Replacing multiple occurrences, i.e., >=2 occurrences with '.'
        text_twitter = re.sub("\n", "", text_twitter)
        text_twitter = re.sub("\n\n", ".", text_twitter)
        text_twitter = text_twitter.strip()

    # Fix contractions
    if condensed:
        text_twitter = cm.fix(text_twitter)
        text_twitter = re.sub("\s\s", "", text_twitter)

    # Remove "ellipsis"
    if ellipsis:
        text_twitter = re.sub("\.{2,}", "", text_twitter)

    # Replace "&" with "and"
    if ampersand:
        text_twitter = text_twitter.replace("&amp", "")

    # Replace "~" with "about"
    if tilde:
        text_twitter = re.sub("~", "about", text_twitter)

    # Remove the special_chars list: [%, ^, *, -, _, +, =, |, \, /, ?]
    if special_chars:
        spec_char_list = ['%', '^', '*', '-', '_', '+', '=', '|', '/', '?']
        sent = ""
        new_sent_tokens = []

        for character in text_twitter:
            if str(character) not in spec_char_list:
                new_sent_tokens.append(character)

        sent = sent.join(new_sent_tokens)
        sent = sent.strip()
        text_twitter = sent

    # Rename $ as dollar
    # if dollar:
    #     text_twitter = re.sub("$", "dollar", text_twitter)

    # Remove brackets and any text enclosed within simple brackets, usually used for acronyms
    if bracketed_phrases:
        text_twitter = re.sub("\(\w+\)", "", text_twitter)

    # If single quotes or double quotes have been used in tweets, encash their meaning for the time being. Don't include any other information
    if quotation_marks:
        text_twitter = re.sub("(\'|\")[a-zA-Z0-9\s+\.]*(\'|\")", "", text_twitter)

    # For the time being, replace commas by "" and semicolons by "."
    if commas_semicols:
        text_twitter = re.sub("\,+", "", text_twitter)
        text_twitter = re.sub("\;+", ".", text_twitter)

    # Resolve '>' and '<'
    # Replace these characters with their respective names
#     if greater_than_less_than:
#         text_twitter = re.sub("<", "is less than", text_twitter)
#         text_twitter = re.sub(">", "is greater than", text_twitter)
#         text_twitter = re.sub("<=", "is less than or equal to", text_twitter)
#         text_twitter = re.sub(">=", "is greater than or equal to", text_twitter)

    # For the time being, replace and interjections with a full stop
    if question_mark_exclaim:
        text_twitter = re.sub("(\?|\!)+", ".", text_twitter)

    # Resolve character encodings
    if character_encodings:
        text_twitter = re.sub("â|€|¦|â|€˜|€™", "", text_twitter)

    # Remove trademark symbol
    if trademark:
        text_twitter = re.sub("\u2122", "", text_twitter)

    return text_twitter

clean_text_twitter = TextCleaner(text_cleaner = cleanTwitter, replace_text = False, save_original = True)

def convert_to_lower(utt: str) -> str:
    """ This function block performs twitter text normalization

        For instance, the different forms of 'hate' are: Hate, HATE, haTE, etc. This function would convert all such occurences to a single canonical form
        """
    exclude_tags_list = ['NN', 'NNS', 'NNP', 'NNPS']  # Check if the attached POS tags are correct or not
    sents = nltk.sent_tokenize(utt)
    modified_sent_ls = []

    for sent in sents: 
        modified_token_ls = []
        words = nltk.word_tokenize(sent)  # Tokenize the sentence and extract POS tags

        words = [lemmatizer.lemmatize(word) for word in words]  # Perform lemmatization if required
        word_pos_tags = nltk.pos_tag(words)

        for (word, tag) in word_pos_tags:
            if tag not in exclude_tags_list or word != "I":
                word = word.lower()
                modified_token_ls.append(word)
                
        modified_token_ls[0] = modified_token_ls[0].capitalize()
                
        utt = " ".join(modified_token_ls)
        utt = utt.strip()
        modified_sent_ls.append(sent)
        
    final_text = " ".join(modified_sent_ls)
    
    return final_text

lowercase_text = TextProcessor(proc_fn = convert_to_lower, output_field = "lowercase_text")

# Use a combination of IDENTITY ATTACK and INSULT parameters to separate MICROAGGRESSIONS from OTHER HATE-SPEECH forms 
def sentiment_analyzer(utt: str) -> dict:
    sentence_ls = nltk.sent_tokenize(utt)
    sentiment_score_ls = []

    for sent in sentence_ls:
        sentiment_score_ls.append(sentiment.polarity_scores(sent))

    pos_score_sum, neu_score_sum, neg_score_sum = 0, 0, 0
    for sentiment_scores in sentiment_score_ls:
        pos_score_sum += sentiment_scores["pos"]
        neu_score_sum += sentiment_scores["neu"]
        neg_score_sum += sentiment_scores["neg"]

    pos_score_sum_avg = round((pos_score_sum / len(sentiment_score_ls)), 3)
    neu_score_sum_avg = round((neu_score_sum / len(sentiment_score_ls)), 3)
    neg_score_sum_avg = round((neg_score_sum / len(sentiment_score_ls)), 3)

    compound_sentiment_scores = {
        "pos": pos_score_sum_avg,
        "neu": neu_score_sum_avg,
        "neg": neg_score_sum_avg
    }
    return compound_sentiment_scores
    
analyze_sentiment = TextProcessor(proc_fn = sentiment_analyzer, input_field="lowercase_text",
                                  output_field="sentiment_polarity")

def modifier_count(utt: str) -> int:   # Calculating less of something isn't always the best indicator. Instead the prevalence of something more than ususal is a better marker. # Optional - Emergency Toolkit  
    """Count modifiers, i.e., adjectives and adverbs in an utterance
    Practically every sentence has modifiers. This function doesn't act as a filter. It is intended to be applied to the entire dataframe
    The function block can detect probable deceptive clues in tweets and reddit posts
    Less usage of descriptive modifiers is a possible clue that the speaker is uncertain in his claims/opinions.  
    This function is more of an emergency toolkit 
    """
    
    adj_pos_tags = ['JJ', 'JJR', 'JJS']  # POS tags describing adjectives
    adv_pos_tags = ['RB', 'RBR', 'RBS']  # POS tags for adverbs
    words = nltk.word_tokenize(utt)
    word_tag_lst = nltk.pos_tag(words)
    mod_count_dict = defaultdict(def_value)
    count_mod_tags = 0
    for (word, tag) in word_tag_lst:
        if tag in adj_pos_tags or tag in adv_pos_tags:
            count_mod_tags += 1
            mod_count_dict[word] += 1
    return {"modifier_count_dict": mod_count_dict, "count_mod_tags": count_mod_tags}

modifier_count = TextProcessor(proc_fn = modifier_count, input_field = "lowercase_text", output_field = "modifier_count")

def hedge_count(utt: str) -> int:
    """ Count the list of all modal verbs that indicate possibility, but not certainty
    The function block can detect probable deceptive clues in tweets and reddit posts
    More usage of uncertain modal verbs is a possible clue that the speaker is uncertain in his utterance
    """
    count_hedges, hedge_count_dict = 0, defaultdict(def_value)
    set_of_hedges_en = ["almost", "apparent", "apparently", "appear", "appeared", "appears", "approximately", "argue",
                        "argued", "argues", "around", "assume", "assumed", "broadly", "certain amount",
                        "certain extent", "certain level", "claim", "claimed", "claims", "doubt", "doubtful",
                        "essentially", "estimate", "estimated", "fairly", "feel", "feels", "felt", "frequently",
                        "from my perspective",
                        "from our perspective", "from this perspective", "generally", "guess", "in general",
                        "in most cases", "in most instances", "in my opinion", "in my view", "in our opinion",
                        "in our view",
                        "in this view", "indicate", "indicated", "indicates", "largely", "likely", "mainly", "may",
                        "maybe", "might", "mostly", "often", "on the whole", "ought", "perhaps", "plausible",
                        "plausibly", "possible",
                        "possibly", "postulate", "postulated", "postulates", "presumable", "presumably", "probable",
                        "probably", "quite", "rather", "relatively", "roughly", "seems", "should", "sometimes",
                        "somewhat", "suggest",
                        "suggested", "suggests", "suppose", "supposed", "supposes", "suspect", "suspects", "tend to",
                        "tended to", "tends to", "think", "thinking", "thought", "to my knowledge", "typical",
                        "typically", "uncertain",
                        "uncertainly", "unclear", "unclearly", "unlikely",
                        "usually"]  # The Hedge word list has been taken from "https://github.com/tslmy/politeness-estimator.git"

    pos_modal_ls = ["shall", "should", "can", "could", "will", "would", "may", "must",
                    "might"]  # List of 9 modal verbs indicating possibility

    hedges_modals = set_of_hedges_en + pos_modal_ls

    words = utt.lower().split(" ")
    for word in words:
        if word in hedges_modals:
            count_hedges += 1
            hedge_count_dict[word] += 1
    return {"hedge_count_dict": hedge_count_dict, "count_hedges": count_hedges}

hedge_modals = TextProcessor(proc_fn = hedge_count, input_field="lowercase_text",
                             output_field="hedge_count")

def group_ref_count(utt: str) -> int: # Include all third-party pronouns as well
    """Count list of group references
    Usage of more self-references along with subjectivity score is a possible indication of deception 
    """
    count_group_ref, group_ref_count_dict = 0, defaultdict(def_value) 
    words = utt.lower().split()
    group_ref = ["we", "our", "ours", "ourselves", "us", "they", "them", "thesmselves", "their", "theirs",
                 "everyone", "everybody"]  # More of it to be included here. Self-referencing pronouns

    for word in words:
        if word in group_ref:
            count_group_ref += 1
            group_ref_count_dict[word] += 1
    return {"group_ref_count_dict": group_ref_count_dict, "count_group_ref": count_group_ref}

group_ref_count = TextProcessor(proc_fn = group_ref_count, input_field = "lowercase_text", output_field = "groupRef_count")

def subjectivity_utterance(utterance: str) -> int:
    """ Textblob subjectivity score
    A higher subjective score indicates personal opinion.
    Low subjective scores could be a possible indicator of deception. To be used along with self references. 
    """
    subjective_lexicon_count, subjective_lexicon_dict, subjectivity_scores = 0, defaultdict(def_value), []
    sents = nltk.sent_tokenize(utterance)
    
    for sent in sents: 
        subjectivity_scores.append(tb(sent).sentiment.subjectivity)
        words = sent.lower().split()
        
        for word in words: 
            if word in subjectivity_clues:
                subjective_lexicon_dict[word] += 1 
                subjective_lexicon_count += 1 
            
    avg_subjectivity_score = round((sum(subjectivity_scores) / len(subjectivity_scores)), 3)
    subjective_details = {
                            "avg_subjectivity_score": avg_subjectivity_score,
                            "subjective_lexicon_count": subjective_lexicon_count, 
                            "subjective_lexicon_dict": subjective_lexicon_dict
                         }
    
    return subjective_details
      
subjectivity = TextProcessor(proc_fn = subjectivity_utterance, input_field = "lowercase_text", output_field = "subjectivity_score")

def count_emojis(utterance: str) -> int: # Optional - Emergency Toolkit - 
    """ Counts the total number of emojis in an utterance
    We don't intend to delete tweets that have emojis. This function doesn't act as a filter. It is intended to be applied to the entire dataframe
    Decide if it is redundant or not - maybe some possible indicators - not the first choice anyhow 
    """
    emot_dict = emotion.emoji(utterance)

    return len(emot_dict['value'])

emoji_count = TextProcessor(proc_fn = count_emojis, input_field = "lowercase_text", output_field = "emoji_count")

def measurePoliteness(utt: str):
  """ Computes politeness indicators in the text. The 9 positive politeness strategies 
  """
  # I don't need to split up the utterances here as I am getting an aggregated count of politeness indicators anyway. 
    
  transformed_utt = politeness.transform_utterance(utt, spacy_nlp=spacy_nlp)
  return transformed_utt.meta['politeness_strategies']

politeness_markers = TextProcessor(proc_fn = measurePoliteness, input_field = "lowercase_text", output_field = "politeness_markers")


In [23]:
# The following pipe would be expanded further 
# Check if all functions from above have been added 
pipe_reddit = ConvokitPipeline ([
                    # ("clean reddit text", clean_text_reddit), - To be done later  
                    ("convert to lowercase", lowercase_text), 
                    ("sentiment analyzer", analyze_sentiment), 
                    ("count modifiers", modifier_count), 
                    ("count hedges and modals", hedge_modals), 
                    ("count group references", group_ref_count),
                    ("subjectivity score", subjectivity),
                    ("politeness indicators", politeness_markers)
])

# The following pipe would be expanded further 
# pipe_twitter = ConvokitPipeline ([
#                     ("extract hashtags", hashtags), 
#                     ("extract reply_to usernames", reply_to_usernames),
#                     ("clean twitter text", clean_text_twitter), 
#                     ("convert to lowercase", lowercase_text), 
#                     ("sentiment analyzer", sentiment_analyzer),
#                     ("count modifiers", modifier_count), - Emergency Toolkit 
#                     ("count hedges and modals", hedge_modals), 
#                     ("count self references", self_ref_count), 
#                     ("subjectivity score", subjectivity_score),     # Desception - to be used with Self-References and
#                     ("politeness indicators", politeness_markers)   # Politeness

# ])

In [24]:
try:
    # Transform the reddit corpus 
    transformed_corpus_reddit = pipe_reddit.transform(reddit_corpus, verbosity = 50)
except Exception as e:
    print(e)
else:
    print("Congratulations, the reddit corpus has been transformed")
    
# try:
#     # Transform the twitter corpus 
#     transformed_twitter_corpus = pipe_twitter.transform(corpus)
# except:
#     print("No defined corpus")
# else:
#     print("Congratulations, the twitter corpus has been transformed")

Congratulations, the reddit corpus has been transformed


In [ ]:
count_mod_tags

## Dump the corpus 

By default, calling **corpus.dump** will write all of these attributes to disk, within the file that stores utterances; later calling **corpus.load** will load all of these attributes back into a new corpus. For big objects like parses, this incurs a high computational burden (especially if in a later use case you might not even need to look at parses).

To avoid this, **corpus.dump** takes an optional argument **fields_to_skip**, which is a **dict of object type ('utterance', 'conversation', 'speaker', 'corpus')** to a list of fields that we do not want to write to disk.

The following call will write the corpus to disk, without any of the preprocessing output we generated above:

In [47]:
# Demonstration of re-dumping of the transformed corpus after first batch of test
try:
    corpus.dump(os.path.basename(OUT_DIR), base_path=os.path.dirname(OUT_DIR), 
            fields_to_skip={'utterance': ['parsed','tagged','clean_text']})    # Just random field names 
    
except:
    print("NO OUTPUT DIRECTORY")

NO OUTPUT DIRECTORY


In [25]:
# Demonstration of re-dumping of the transformed corpus after first batch of test
try:
    CORPUS_NAME = "tr_fc(12-23)_27p_1"
    OUTPUT_DIR = "C:\Sagar Study\ML and Learning\CP Sem-8\Data\Reddit\saved-corpora\sanskar_transformed_corpora"
    transformed_corpus_reddit.dump(CORPUS_NAME, base_path=OUTPUT_DIR)
    
except:
    print("NO OUTPUT DIRECTORY")

In [26]:
 transformed_corpus_reddit.random_utterance()

Utterance({'obj_type': 'utterance', 'meta': {'lowercase_text': "I mean you're certainly a mega douche for what it's worth", 'sentiment_polarity': {'pos': 0.291, 'neu': 0.541, 'neg': 0.169}, 'modifier_count': {'modifier_count_dict': defaultdict(<function def_value at 0x0000024FCABB7670>, {'certainly': 1, 'mega': 1, 'worth': 1}), 'count_mod_tags': 3}, 'hedge_count': {'hedge_count_dict': defaultdict(<function def_value at 0x0000024FCABB7670>, {}), 'count_hedges': 0}, 'groupRef_count': {'group_ref_count_dict': defaultdict(<function def_value at 0x0000024FCABB7670>, {}), 'count_group_ref': 0}, 'subjectivity_score': {'avg_subjectivity_score': 0.453, 'subjective_lexicon_count': 3, 'subjective_lexicon_dict': defaultdict(<function def_value at 0x0000024FCABB7670>, {'mean': 1, 'certainly': 1, 'worth': 1})}, 'politeness_markers': {'feature_politeness_==Please==': 0, 'feature_politeness_==Please_start==': 0, 'feature_politeness_==HASHEDGE==': 0, 'feature_politeness_==Indirect_(btw)==': 0, 'feature

For attributes we want to keep around, but that we don't want to read and write to disk in a big batch with all the other corpus data, corpus.dump_info will dump fields of a Corpus object into separate files. This takes the following arguments as input:

* obj_type: which type of Corpus object you're dealing with
* fields: a list of the fields to write
* dir_name: which directory to write to; by default will write to the directory you read the corpus from

This function will write each field in fields to a separate file called **info.<field>.jsonl** where each line of the file is a json-serialized dict: {"id": <ID of object>, "value": <object.get_info(field)>}.